## Setup

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import ipywidgets as widgets
from ipywidgets import interact, fixed

plt.rcParams["figure.figsize"] = (14, 4)

## Load

In [2]:
m4_info = pd.read_csv("../data/M4-info.csv")
m4_info.columns = ["id", "category", "freq", "fh", "sp", "start_date"]
m4_info["start_date"] = pd.to_datetime(m4_info["start_date"])
m4_info

,id,category,freq,fh,sp,start_date
0,Y1,Macro,1,6,Yearly,1979-01-01 12:00:00
1,Y2,Macro,1,6,Yearly,1979-01-01 12:00:00
2,Y3,Macro,1,6,Yearly,1979-01-01 12:00:00
3,Y4,Macro,1,6,Yearly,1979-01-01 12:00:00
4,Y5,Macro,1,6,Yearly,1979-01-01 12:00:00
...,...,...,...,...,...,...
99995,H410,Other,24,48,Hourly,2017-01-01 12:00:00
99996,H411,Other,24,48,Hourly,2017-01-01 12:00:00
99997,H412,Other,24,48,Hourly,2017-01-01 12:00:00
99998,H413,Other,24,48,Hourly,2017-12-06 12:00:00


In [3]:
daily_train = pd.read_csv("../data/Train/Daily-train.csv")
daily_train

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V9911,V9912,V9913,V9914,V9915,V9916,V9917,V9918,V9919,V9920
0,D1,1017.10,1019.30,1017.00,1019.20,1018.70,1015.60,1018.50,1018.30,1018.40,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,D2,2793.70,2793.80,2803.70,2805.80,2802.30,2795.00,2806.40,2782.20,2780.30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,D3,1091.30,1088.50,1085.70,1082.90,1080.10,1077.30,1074.50,1071.70,1068.90,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,D4,1092.00,1078.00,1064.00,1050.00,1036.00,1022.00,1008.00,1092.00,1078.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,D5,2938.63,2956.44,2964.41,2972.41,3014.97,3014.23,3024.08,3031.97,3062.70,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4222,D4223,10000.00,3230.00,3238.00,3234.00,3225.00,3209.00,3207.00,3203.00,3204.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4223,D4224,10000.00,3997.00,4000.00,3973.00,3963.00,3984.00,3956.00,3896.00,3869.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4224,D4225,352000.00,4186.00,4171.00,4196.00,4184.00,4192.00,4192.00,4178.00,4146.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4225,D4226,994.16,991.55,976.97,993.15,990.79,992.49,1003.51,1024.51,1021.65,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
m4_info.pivot_table(values=["freq", "fh"], index="sp")

,fh,freq
sp,,
Daily,14,1
Hourly,48,24
Monthly,18,12
Quarterly,8,4
Weekly,13,1
Yearly,6,1


In [5]:
m4_info.pivot_table(index="sp", columns="category", aggfunc="size").fillna(0).astype("int")

category,Demographic,Finance,Industry,Macro,Micro,Other
sp,,,,,,
Daily,10,1559,422,127,1476,633
Hourly,0,0,0,0,0,414
Monthly,5728,10987,10017,10016,10975,277
Quarterly,1858,5305,4637,5315,6020,865
Weekly,24,164,6,41,112,12
Yearly,1088,6519,3716,3903,6538,1236


## Prepare

In [6]:
def build_timestamp(ts):
    start = ts["start_date"].iloc[0]
    ts = ts.drop(columns=["start_date"])
    ts["ds"] = pd.date_range(start=start, periods=len(ts))
    return ts

In [7]:
def initial_preprocessing(df):
    df = df.copy()
    df = df.melt(id_vars="V1", var_name="t", value_name="y")
    df = df.rename(columns={"V1": "id"})
    df = df.merge(m4_info[["id", "category", "start_date"]], how="left", on="id")
    df["id"] = df["id"].str[1:].astype("int")
    df["t"] = df["t"].str[1:].astype("int") - 1
    df["category"] = df["category"].astype("category")
    df = df.groupby("id", group_keys=False).apply(lambda x: x.loc[:x.y.last_valid_index()])
    df = df.groupby("id").apply(build_timestamp)
    df = df[["id", "ds", "t", "y", "category"]].copy()
    return df

In [8]:
daily_train_base = initial_preprocessing(daily_train)
daily_train_base

,id,ds,t,y,category
0,1,1994-03-01 12:00:00,1,1017.1,Macro
4227,1,1994-03-02 12:00:00,2,1019.3,Macro
8454,1,1994-03-03 12:00:00,3,1017.0,Macro
12681,1,1994-03-04 12:00:00,4,1019.2,Macro
16908,1,1994-03-05 12:00:00,5,1018.7,Macro
...,...,...,...,...,...
18750971,4227,2012-04-22 12:00:00,4436,15111.5,Other
18755198,4227,2012-04-23 12:00:00,4437,15154.0,Other
18759425,4227,2012-04-24 12:00:00,4438,15208.6,Other
18763652,4227,2012-04-25 12:00:00,4439,15108.0,Other


In [9]:
daily_train_base.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9964658 entries, 0 to 18767879
Data columns (total 5 columns):
 #   Column    Dtype         
---  ------    -----         
 0   id        int32         
 1   ds        datetime64[ns]
 2   t         int32         
 3   y         float64       
 4   category  category      
dtypes: category(1), datetime64[ns](1), float64(1), int32(2)
memory usage: 313.6 MB


In [10]:
daily_train_base.isna().sum()

id          0
ds          0
t           0
y           0
category    0
dtype: int64

In [11]:
daily_train_base.to_csv("../data/processed/daily-train-base.csv", index=False)